# Product Search: Best Buy & Walmart

This code base combines product search from Walmart and Best Buy. Early versions were developed separately.  

The search versions labeld 0.n were shown just for reference.

Purpose of Code:
1. Search product buy keyword (e.g. laptop) at Best Buy and Walmart

---Iterative search loop
2. Allow programmer to select the product parameters to store
3. Return a complete list of all products within the query
4. Rename the product parameters to a common columns names
5. Run a regular expression loop to pull out screensize and store in a column 
3. Write results to a csv file (or json in earlier iterations of the code)


Problems Solved:
1. Best Buy allows you to download and store entire product database. Walmart does not
2. Complete Product Inventory for Walmart.

---Walmart has a much larger product inventory per category in several areas. 

---For example, 20,000 laptops with a limit of 1000 for paginated search

--We got around this by running a loop to search the entire price range and append results to a file

3. Walmart json not formated properly

---When you append successive searches in Walmart, the json is not structured properly to read back
into a dataframe

---This method overcomes that issue by saving each search in a separate csv file, then combining

4. Product information missing, for example screen size of TVs

---This method contains regular expression to capture screen size

In [1]:
import requests
import json
import csv
import pandas as pd
import time
import re
from pathlib import Path

def print_r(text):
    print(json.dumps(text, indent=4, sort_keys=False))


In [2]:
api_key = "INSERT API KEY"

# base search URL from Walmart
url = "http://api.walmartlabs.com/v1/search" 

# we will run a search based on price
# price range will be based in facet.range

#initialize faceRange, this will change below
priceLow = 151
priceHigh = 200
facetRange = f"price:[{priceLow} TO {priceHigh}]"


params = {
    "apiKey": api_key,
    "format": "json", # json|xml
    "numItems": 25,
    "sort": "bestseller", # relevance, price, title, bestseller, customerRating, new
    "order": "desc", # asc, desc
    "facet":"on",
    "facet.range":"price:[151 TO 200]",
    #"start": 1,
    #"categoryId": "3944_3951_1089430_1230093_1230101"
}
params["query"] = "laptop"

# Walmart Search

In [211]:
# this is the basic search function for walmart

def walmart_search(q):
    params["query"] = q
    walmart_data = requests.get(url, params=params)
    walmart_url = walmart_data.url
    try:
        walmart_data = walmart_data.json()
    except ValueError:
        print("Decoding JSON has failed in:")
        print(walmart_url)
        walmart_data = {
            "DecodeERROR":1
        }
    #print(walmart_url)
    return walmart_data


In [270]:
#print_r(params)
params["start"] = 1


query = "Laptop"
startpage = 0
maxpages = 180
defer = 50

query = "TV"
startpage = 0
maxpages = 80
defer = 100

query = "Phone"
startpage = 2
maxpages = 80
defer = 50

#out_file = open("walmart_"+query+"_data_file3.csv","w")
out_file = "Walmart_datasets/walmart_"+query+"_data_file.csv"
all_data =[]

for xx in range(startpage,maxpages):
    strt_price = xx*defer+1
    end_price = xx*defer+defer
    out_file2 = "output/walmart_"+query+"_"+str(strt_price)+"-"+str(end_price)+"_data_file.csv"
    my_file = Path(out_file2)
    if my_file.is_file():
        # file exists
        #print("file exists")
        read_df = pd.read_csv(out_file2)
        #read_df.head()
        for index, row in read_df.iterrows():
            item_data = {}
            item_data["itemId"]            = row["itemId"]
            item_data["parentItemId"]      = row["parentItemId"]
            item_data["name"]              = row["name"]
            item_data["salePrice"]         = row["salePrice"]
            item_data["upc"]               = row["upc"]
            item_data["categoryPath"]      = row["categoryPath"]
            item_data["numReviews"]        = row["numReviews"]
            item_data["customerRating"]    = row["customerRating"]
            item_data["thumbnailImage"]    = row["thumbnailImage"]
            item_data["mediumImage"]       = row["mediumImage"]
            item_data["largeImage"]        = row["largeImage"]
            item_data["categoryNode"]      = row["categoryNode"]
            item_data["offerType"]         = row["offerType"]
            m = re.findall(r'[0-9\.]+"', row["name"])
            #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
            if len(m) >0:
                m = m[0].replace("\"","")
            else:
                m = 0
            #print(m)
            item_data["screensize"]    = m
            all_data.append(item_data)
    else:
        prices = "price:["+str(strt_price)+" TO "+str(end_price)+"]"
        all_data2 =[]
        params["facet.range"] = prices
        #print(params)
        ret = walmart_search(query)
        if "totalResults" not in ret:
            continue
        totalResults =ret["totalResults"]
        if totalResults==0:
            continue
        pages2 = totalResults/25+1
        pages = int(round(pages2,0))
        if pages2>pages:
            pages +=1
        #print(pages)
        print("Looking for ("+query+") price range ("+str(strt_price)+","+str(end_price)+"), Total results ("+str(totalResults)+") in ("+str(pages-1)+") Pages")
        for i in range(1,pages):
            time.sleep(1)
            strt = (i*25) - 24;
            if strt<=1000:
                params["start"] = strt
                try:
                    if strt!=1:
                        ret = walmart_search(query)
                    if "DecodeERROR" not in ret:
                        if ret["numItems"]>0:
                            #print_r(ret)
                            #print(ret["items"][0]["name"])
                            for x in ret["items"]:
                                try:
                                    if "numReviews" not in x:
                                        x["numReviews"]=0
                                        x["customerRating"]="0.0"
                                    if "largeImage" not in x:
                                        x["largeImage"]="0.0"
                                    if "salePrice" in x and "upc" in x:
                                        if "\\" not in x["name"]:
                                            item_data = {}
                                            item_data["itemId"] = x["itemId"]
                                            item_data["parentItemId"] = x["parentItemId"]
                                            try:
                                            #    out_body.append(str(x["name"]))
                                                item_data["name"] = x["name"]
                                            except (KeyError, IndexError):
                                            #    out_body.append(str("Error with the name"))
                                                item_data["name"] = "Error with the name"

                                            item_data["salePrice"] = x["salePrice"]
                                            item_data["upc"] = x["upc"]
                                            item_data["categoryPath"] = x["categoryPath"]
                                            item_data["numReviews"] = x["numReviews"]
                                            item_data["customerRating"] = x["customerRating"]
                                            item_data["thumbnailImage"] = x["thumbnailImage"]
                                            item_data["mediumImage"] = x["mediumImage"]
                                            item_data["largeImage"] = x["largeImage"]
                                            item_data["categoryNode"] = x["categoryNode"]
                                            item_data["offerType"] = x["offerType"]
                                            m = re.findall(r'[0-9\.]+"', x["name"])
                                            #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
                                            if len(m) >0:
                                                m = m[0].replace("\"","")
                                            else:
                                                m = 0
                                            #print(m)
                                            item_data["screensize"]    = m
                                            all_data.append(item_data)
                                            all_data2.append(item_data)
                                except (KeyError, IndexError):
                                    print("Missing field/result... skipping.")

                except (KeyError, IndexError):
                    print("Missing field/result... skipping.")
            #print(row["upc"], index)
        results_pd2 = pd.DataFrame(all_data2)
        results_pd2.to_csv(out_file2, sep=',')

print("----------------------------------")
print("We Finished, but not yet")


----------------------------------
We Finished, but not yet


In [272]:
#print(all_data)
results_pd = pd.DataFrame(all_data)
results_pd.to_csv(out_file, sep=',')
results_pd


,categoryNode,categoryPath,customerRating,itemId,largeImage,mediumImage,name,numReviews,offerType,parentItemId,salePrice,screensize,thumbnailImage,upc
0,1085632_1094344_2365074,Seasonal/Father's Day/Father's Day Electronics,0.0,712004929,https://i5.walmartimages.com/asr/f9ddc826-fa3d...,https://i5.walmartimages.com/asr/f9ddc826-fa3d...,ZOJI-Z7 4G Mobile Cell Phone Quad Core Fingerp...,0,ONLINE_ONLY,712004929,127.64,0,https://i5.walmartimages.com/asr/f9ddc826-fa3d...,192278851443
1,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,774166579,https://i5.walmartimages.com/asr/44d9a4d9-7b01...,https://i5.walmartimages.com/asr/44d9a4d9-7b01...,"""Vtech CS6909 (4-Pack) Accessory Handset for C...",0,ONLINE_ONLY,774166579,104.85,0,https://i5.walmartimages.com/asr/44d9a4d9-7b01...,657379498404
2,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,138382833,https://i5.walmartimages.com/asr/289b3510-23a1...,https://i5.walmartimages.com/asr/289b3510-23a1...,Escence 2 SIP Account IP Phone,0,ONLINE_ONLY,138382833,149.00,0,https://i5.walmartimages.com/asr/289b3510-23a1...,700443222761
3,1229749_1070964_3977,Office/Office Technology/Phones & Accessories/...,0.0,519023948,https://i5.walmartimages.com/asr/087c8a72-2c70...,https://i5.walmartimages.com/asr/087c8a72-2c70...,IP PHONE 7841 FOR 3RD PARTY CALL CONTROL,0,ONLINE_ONLY,519023948,140.70,0,https://i5.walmartimages.com/asr/087c8a72-2c70...,882658829802
4,1105910_1072335_1231295,Cell Phones/No-Contract Phones & Plans/Shop Ph...,0.0,460217854,https://i5.walmartimages.com/asr/f1488e6d-5a56...,https://i5.walmartimages.com/asr/f1488e6d-5a56...,Indigi® 5.6inch 2018 4G LTE Unlocked Android 6...,0,ONLINE_ONLY,460217854,129.99,0,https://i5.walmartimages.com/asr/f1488e6d-5a56...,798711873555


# Best Buy Search

In [225]:
bestbuy_api_key = "INSERT API KEY"
bestbuyUrl = "https://api.bestbuy.com/v1/products"

show_arr=[
    #"accessories.sku",
    #"color",
    "condition",
    "customerReviewAverage",
    "customerReviewCount",
    "customerTopRated",
    #"depth",
    "description",
    "department",
    "departmentId",
    #"details.name",
    #"details.value",
    "digital",
    #"features.feature",
    "format",
    "height",
    "salePrice",
    #"includedItemList.includedItem",
    "longDescription",
    #"longDescriptionHtml",
    "manufacturer",
    "modelNumber",
    "name",
    "preowned",
    #"productVariations.sku",
    "quantityLimit",
    "releaseDate",
    "shortDescription",
    "shortDescriptionHtml",
    "sku",
    "upc",
    "warrantyLabor",
    "inStoreAvailability",
    "onlineAvailability",
    "warrantyParts",
    "weight",
    "width"
    ,"image"
    ,"largeImage"
    ,"thumbnailImage"
]
params2 = {
    "apiKey": bestbuy_api_key,
    "page": 1,
    "show": ",".join(show_arr),
    "pageSize": 100,
    "format": "json" # json|xml
}
params2["query"] = "ipad"
print(bestbuyUrl)


https://api.bestbuy.com/v1/products


In [226]:
def bestbuy_search(q):
    urlquery = bestbuyUrl + "(search=" + q + ")"
    bestbuy_data = requests.get(urlquery, params=params2)
    bestbuy_url = bestbuy_data.url
    bestbuy_data = bestbuy_data.json()
    #print_r(bestbuy_url)
    return bestbuy_data

In [269]:
query = "TV"
query = "Phone"
query = "Laptop"



out_file = "Walmart_datasets/bestbuy_"+query+"_data_file.csv"
all_data=[]
ret = bestbuy_search(query)
if "total" in ret:
    #print_r(ret)
    total = ret["total"]
    totalPages = ret["totalPages"]
    prices = "price:["+str(strt_price)+" TO "+str(end_price)+"]"
    all_data2 =[]
    for xx in range(1,(totalPages+1)):
    #for xx in range(1,(1+1)):
        page = xx
        out_file2 = "output/bestbuy_"+query+"_"+str(page)+"_data_file.csv"
        my_file = Path(out_file2)
        if my_file.is_file():
            # file exists
            #print("file exists")
            read_df = pd.read_csv(out_file2)
            #read_df.head()
            for index, row in read_df.iterrows():
                item_data = {}
                item_data["itemId"]         = row["itemId"]
                item_data["department"]   = row["department"]
                item_data["name"]           = row["name"]
                item_data["salePrice"]      = row["salePrice"]
                item_data["upc"]            = row["upc"]
                item_data["departmentId"]   = row["departmentId"]
                item_data["numReviews"]     = row["numReviews"]
                item_data["customerRating"] = row["customerRating"]
                item_data["thumbnailImage"] = row["thumbnailImage"]
                item_data["mediumImage"]    = row["mediumImage"]
                item_data["largeImage"]     = row["largeImage"]
                #item_data["categoryNode"]   = row["categoryNode"]
                item_data["inStoreAvailability"]      = row["inStoreAvailability"]
                item_data["onlineAvailability"]      = row["onlineAvailability"]
                item_data["manufacturer"]      = row["manufacturer"]
                item_data["quantityLimit"]      = row["quantityLimit"]
                item_data["manufacturer"]      = row["manufacturer"]
                m = re.findall(r'[0-9\.]+"', row["name"])
                #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
                if len(m) >0:
                    m = m[0].replace("\"","")
                else:
                    m = 0
                #print(m)
                item_data["screensize"]    = m
                all_data.append(item_data)
        else:
            all_data2 =[]
            params2["page"] = page
            #print(params2)
            time.sleep(1)
            if(page>1):
                ret = bestbuy_search(query)
            print("Looking for ("+query+") page ("+str(page)+"), Total results ("+str(total)+") in ("+str(totalPages)+") Pages")
            #try:
            if total>0:
                #print_r(ret)
                #print(ret["items"][0]["name"])
                #print_r(ret)
                for x in ret["products"]:
                    #try:
                    item_data = {}
                    item_data["itemId"]         = x["sku"]
                    item_data["department"]   = x["department"]
                    item_data["name"]           = x["name"]
                    item_data["salePrice"]      = x["salePrice"]
                    item_data["upc"]            = x["upc"]
                    item_data["departmentId"]   = x["departmentId"]
                    item_data["numReviews"]     = x["customerReviewCount"]
                    item_data["customerRating"] = x["customerReviewAverage"]
                    item_data["thumbnailImage"] = x["thumbnailImage"]
                    item_data["mediumImage"]    = x["image"]
                    item_data["largeImage"]     = x["largeImage"]
                    #item_data["categoryNode"]   = x["categoryNode"]
                    item_data["inStoreAvailability"]      = x["inStoreAvailability"]
                    item_data["onlineAvailability"]      = x["onlineAvailability"]
                    item_data["manufacturer"]      = x["manufacturer"]
                    item_data["quantityLimit"]      = x["quantityLimit"]
                    item_data["manufacturer"]      = x["manufacturer"]
                    m = re.findall(r'[0-9\.]+"', x["name"])
                    if len(m) >0:
                        m = m[0].replace("\"","")
                    else:
                        m = 0
                    #print(m)
                    item_data["screensize"]    = m
                    #item_data["screensize"]    = re.findall(r'[0-9\.]+"', x["name"])[0].replace("\"","")
                    all_data.append(item_data)
                    all_data2.append(item_data)
                    #except (KeyError, IndexError):
                    #    print("Missing field/result... skipping.")

            #except (KeyError, IndexError):
                #print("Missing field/result... skipping.")
                #print(row["upc"], index)
            results_pd2 = pd.DataFrame(all_data2)
            results_pd2.to_csv(out_file2, sep=',')

print("----------------------------------")
print("We Finished, but not yet")


----------------------------------
We Finished, but not yet


In [264]:
results_pd = pd.DataFrame(all_data)
results_pd.to_csv(out_file, sep=',')
results_pd

read_df = pd.read_csv(out_file)
all_data2=[]
for index, row in read_df.iterrows():
    #print_r(row)
    item_data = {}
    item_data["itemId"]         = row["itemId"]
    item_data["department"]   = row["department"]
    item_data["name"]           = row["name"]
    item_data["salePrice"]      = row["salePrice"]
    item_data["upc"]            = row["upc"]
    item_data["departmentId"]   = row["departmentId"]
    item_data["numReviews"]     = row["numReviews"]
    item_data["customerRating"] = row["customerRating"]
    item_data["thumbnailImage"] = row["thumbnailImage"]
    item_data["mediumImage"]    = row["mediumImage"]
    item_data["largeImage"]     = row["largeImage"]
    #item_data["categoryNode"]   = row["categoryNode"]
    item_data["inStoreAvailability"]      = row["inStoreAvailability"]
    item_data["onlineAvailability"]      = row["onlineAvailability"]
    item_data["manufacturer"]      = row["manufacturer"]
    item_data["quantityLimit"]      = row["quantityLimit"]
    item_data["manufacturer"]      = row["manufacturer"]
    #m = re.search(r'[0-9\.]+"', row["name"])
    m = re.findall(r'[0-9\.]+"', row["name"])
    #item_data["screensize"]    = re.findall(r'[0-9\.]+"', row["name"])[0].replace("\"","")
    if len(m) >0:
        m = m[0].replace("\"","")
    else:
        m = 0
    #print(m)
    item_data["screensize"]    = m
    all_data2.append(item_data)
    #print(item_data)

results_pd2 = pd.DataFrame(all_data2)
results_pd2.to_csv(out_file, sep=',')
results_pd2.head()


,customerRating,department,departmentId,inStoreAvailability,itemId,largeImage,manufacturer,mediumImage,name,numReviews,onlineAvailability,quantityLimit,salePrice,screensize,thumbnailImage,upc
0,NaN,VIDEO/COMPACT DISC,8.0,False,13690446,https://img.bbystatic.com/BestBuy_US/images/pr...,NaN,https://img.bbystatic.com/BestBuy_US/images/pr...,Ohne Filter - Musik Pur: Go Jazz All Stars in ...,NaN,True,3.0,14.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,7.077877e+11
1,NaN,VIDEO/COMPACT DISC,8.0,False,13691837,NaN,NaN,NaN,ANDY GRIFFITH SHOW,NaN,False,3.0,6.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,4.001369e+11
2,NaN,VIDEO/COMPACT DISC,8.0,False,13692113,NaN,NaN,NaN,LUCY SHOW,NaN,False,3.0,6.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,4.001369e+11
3,NaN,VIDEO/COMPACT DISC,8.0,False,13697127,https://img.bbystatic.com/BestBuy_US/images/pr...,NaN,https://img.bbystatic.com/BestBuy_US/images/pr...,Ohne Filter - Musik Pur: Sananda Maitreya (Ter...,NaN,True,3.0,14.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,7.077877e+11
4,NaN,VIDEO/COMPACT DISC,8.0,False,13703218,https://img.bbystatic.com/BestBuy_US/images/pr...,NaN,https://img.bbystatic.com/BestBuy_US/images/pr...,NOVA: Mars - Dead or Alive [DVD],NaN,True,3.0,13.99,0,https://img.bbystatic.com/BestBuy_US/images/pr...,7.834214e+11


In [ ]:
bestbuy_api_key = "INSERT API KEY"
bestbuyUrl = "https://api.bestbuy.com/v1/categories"

params2 = {
    "apiKey": bestbuy_api_key,
    "page": 1,
    #"show": ",".join(show_arr),
    "pageSize": 100,
    "format": "json" # json|xml
}
#params2["query"] = "ipad"
def bestbuy_categories():
    urlquery = bestbuyUrl
    bestbuy_data = requests.get(urlquery, params=params2)
    bestbuy_url = bestbuy_data.url
    bestbuy_data = bestbuy_data.json()
    #print_r(bestbuy_url)
    return bestbuy_data
print(bestbuyUrl)
ret = bestbuy_categories()
print_r(ret)


In [ ]:
out_file = "BestBuy/bestbuy_categories_data_file.csv"
all_data=[]
ret = bestbuy_categories()
if "total" in ret:
    #print_r(ret)
    total = ret["total"]
    totalPages = ret["totalPages"]
    prices = "price:["+str(strt_price)+" TO "+str(end_price)+"]"
    all_data2 =[]
    for xx in range(1,(totalPages+1)):
    #for xx in range(1,(1+1)):
        page = xx
        all_data2 =[]
        params2["page"] = page
        #print(params2)
        time.sleep(1)
        if(page>1):
            ret = bestbuy_categories()
        print("Looking for categories page ("+str(page)+"), Total results ("+str(total)+") in ("+str(totalPages)+") Pages")
        for x in ret["categories"]:
            categories[x["id"]] = {
                "name":x["name"],
                "subCategories":{}
            }
            if "subCategories" in x:
                for i in x["subCategories"]:
                    categories[x["id"]]["subCategories"][i["id"]]=i["name"]

print("----------------------------------")
print("We Finished, but not yet")

results_pd2 = pd.DataFrame(categories)
#results_pd2.to_csv(out_file, sep=',')
results_pd2.head()


In [ ]:
print_r(categories)

In [ ]:
categories = {}
for x in ret["categories"]:
    categories[x["id"]] = {
        "name":x["name"],
        "subCategories":{}
    }
    if "subCategories" in x:
        for i in x["subCategories"]:
            categories[x["id"]]["subCategories"][i["id"]]=i["name"]

In [ ]:
print_r(categories)